In [25]:
import pandas as pd
import numpy as np
import io
import re
from typing import TypedDict, List
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END

In [26]:
# ==========================================
# 1. 环境配置与数据加载
# ==========================================
# 链接本地 LM Studio
llm = ChatOpenAI(
    base_url="http://127.0.0.1:1234/v1",
    api_key="not-needed",
    model_name="local-model",
    temperature=0.2 # 稍微带点随机性以探索不同特征
)

# 模拟读取数据 (实际运行时请改为 pd.read_csv('train.csv'))
# 确保你的 train.csv 放在同级目录下
try:
    df_train_raw = pd.read_csv('./train.csv')
except:
    print("未找到 train.csv，请确保文件存在。")
    exit()

In [27]:
# ==========================================
# 2. 评估函数 (MAP@3)
# ==========================================
def map3_score(y_true, y_probs, labels):
    top3_idx = np.argsort(y_probs, axis=1)[:, -3:][:, ::-1]
    score = 0.0
    y_true = y_true.values if hasattr(y_true, 'values') else y_true
    for i, true_val in enumerate(y_true):
        prediction_list = labels[top3_idx[i]]
        for j, pred in enumerate(prediction_list):
            if pred == true_val:
                score += 1.0 / (j + 1)
                break
    return score / len(y_true)

In [ ]:
# ==========================================
# 3. LangGraph 状态与节点
# ==========================================
class AgentState(TypedDict):
    code: str
    current_score: float
    feedback: str
    iteration_count: int
    best_code: str


In [ ]:
def feature_engineer_node(state: AgentState):
    # 明确定义数值列
    num_cols = "['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous']"
    cat_cols = "['Soil Type', 'Crop Type']"
    
    error_feedback = ""
    if state.get("is_fix_needed", False):
        error_feedback = f"\n⚠️ 注意：上一轮代码报错了，请修复：{state['feedback']}"

    prompt = f"""你是一个高级特征工程专家。
目标：通过对数值型特征进行基本运算（+,-,*,/）生成高阶因子。
{error_feedback}

数据列：
- 数值列：{num_cols}
- 类别列：{cat_cols}

任务要求：
1. 编写函数 `transform_data(df)`。
2. **笛卡尔积特征**：将 {cat_cols} 中的两列进行字符串拼接，生成一个新列 'Soil_Crop_Combined'。
3. **数值因子生成**：从数值列中任选 2-3 个列，通过加减乘除组合生成至少 5 个新因子。
   示例思路：
   - 养分占比：Nitrogen / (Nitrogen + Phosphorous + Potassium)
   - 环境压力：(Temparature * Humidity) / Moisture
   - 养分差异：Nitrogen - Phosphorous
4. **语法规范**：
   - 使用 `df['new_col'] = ...` 语法。
   - 必须处理除零风险（加上 1e-6，例如：a / (b + 1e-6)）。
   - 严禁输出任何 Markdown 标签或解释文字。

代码模板：
import pandas as pd
import numpy as np

def transform_data(df):
    # 1. 类别笛卡尔积
    df['Soil_Crop_Combined'] = df['Soil Type'].astype(str) + "_" + df['Crop Type'].astype(str)
    
    # 2. 数值基本运算因子
    # 在此编写你的逻辑...
    return df
"""
    response = llm.invoke(prompt)
    clean_code = re.sub(r'```python|```', '', response.content).strip()
    return {"code": clean_code}

In [ ]:

def evaluation_node(state: AgentState):
    global df_train_raw
    code = state['code']
    
    # --- 调试：打印 LLM 生成的代码 ---
    print("\n[Node: Evaluate] 收到生成的代码：")
    print("-" * 20)
    print(code)
    print("-" * 20)
    
    try:
        # 动态执行
        exec_globals = {"pd": pd, "np": np}
        exec(code, exec_globals)
        
        if 'transform_data' not in exec_globals:
            raise ValueError("函数 transform_data 未在代码中定义")
            
        transform_fn = exec_globals['transform_data']
        
        # 应用特征
        df = df_train_raw.copy()
        df = transform_fn(df)
        
        # 预处理
        for col in df.select_dtypes(include=['object']).columns:
            if col != 'Fertilizer Name':
                df[col] = df[col].astype('category').cat.codes
        
        X = df.drop(['id', 'Fertilizer Name'], axis=1, errors='ignore')
        y = df['Fertilizer Name']
        
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
        
        model = LGBMClassifier(n_estimators=100, learning_rate=0.1, verbose=-1)
        model.fit(X_train, y_train)
        
        y_probs = model.predict_proba(X_val)
        labels = model.classes_
        score = map3_score(y_val, y_probs, labels)
        
        importances = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)
        top_f = importances.head(3).index.tolist()
        
        feedback = f"成功！分数: {score:.4f}。重要特征: {top_f}"
        print(f"✅ 评估完成。分数: {score:.4f}")
        
        new_best_code = state.get('best_code', "")
        if score > state['current_score']:
            new_best_code = code
            
        return {
            "current_score": score, 
            "feedback": f"成功！分数: {score:.4f}", 
            "is_fix_needed": False  # 标记：不需要修复
        }

    except Exception as e:
        error_msg = f"❌ 报错: {str(e)}"
        return {
            "feedback": error_msg, 
            "is_fix_needed": True   # 标记：需要重写代码
        }

In [29]:
# ==========================================
# 4. 构建图逻辑
# ==========================================
def should_continue(state: AgentState):
    # 如果代码报错，强制返回 engineer 节点重试
    if state.get("is_fix_needed", False):
        print("⚠️ 检测到代码错误，正在请求 Agent 修复...")
        return "engineer"
    
    # 如果迭代次数到上限或分数达标，结束
    if state["iteration_count"] >= 5 or state["current_score"] > 0.98:
        return END
    
    # 正常进入下一轮特征探索
    return "engineer"

workflow = StateGraph(AgentState)
workflow.add_node("engineer", feature_engineer_node)
workflow.add_node("evaluate", evaluation_node)
workflow.set_entry_point("engineer")
workflow.add_edge("engineer", "evaluate")
workflow.add_conditional_edges("evaluate", should_continue)
workflow.add_conditional_edges(
    "evaluate", 
    should_continue,
    {
        "engineer": "engineer", # 报错或继续探索都回 engineer
        END: END
    }
)

ValueError: Branch with name `should_continue` already exists for node `evaluate`

In [ ]:
# ==========================================
# 5. 执行并实时打印每个节点的内容
# ==========================================
app = workflow.compile()

initial_state = {
    "code": "",
    "current_score": 0.0,
    "feedback": "请开始第一轮探索。",
    "iteration_count": 0,
    "best_code": ""
}

print("🚀 Agent 开始工作，正在实时流式输出节点状态...\n")

# 使用 stream 模式查看每一个步骤
for output in app.stream(initial_state):
    # output 的格式是 { "节点名称": { "状态更新内容" } }
    for node_name, state_update in output.items():
        print(f"标注节点: [{node_name}]")
        
        if node_name == "engineer":
            print("📝 LLM 生成的代码片段 (前 100 字符):")
            print(state_update['code'][:100] + "...")
            
        elif node_name == "evaluate":
            print(f"📊 评估结果: {state_update.get('feedback', '无反馈')}")
            print(f"🏆 当前最佳分数: {state_update.get('current_score', 0.0)}")
            
        print("-" * 40)

# 最后打印最终结果
final_state = app.get_state(config={}).values # 获取最后状态（取决于具体版本，可用 invoke 的结果代替）